# Database interface for Arena output log files.

## Import libraries

In [1]:
# import database SQLite3. It is an inbuilt DBMS inside Python.
import sqlite3
import pandas as pd
import numpy as np

## Arena Tracelog Databse Interface

### Main fnuction to read and insert data


In [2]:
def dbwrite_real_arenatracelog(table_name,file_path,database_path):
    # reading the text file as a space separated csv file into pandas dataframe.
    data = pd.read_csv(file_path, delimiter='\s+', header=None, index_col=False)

    # data preparation 
    # indexing of dataframe starts at 0.
    timestamp=data[0].values.astype(int).tolist()
    machine_num=data[2].values
    machine_id=np.char.add("Machine ", machine_num.astype(str)).tolist()
    activity_type=data[3].values.tolist()
    part_id=data[4].values.astype(int).tolist()
    queue_num=data[6].values
    queue=np.char.add("Queue ", queue_num.astype(str)).tolist()

    # zip all the list created above into a single list
    new_data=list(zip(timestamp, machine_id, activity_type, part_id, queue))

    # create table. Also creates database if it doesnt exist.
    with sqlite3.connect(database_path) as realdb:
        realdb.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            event_id INTEGER PRIMARY KEY,
            timestamp INTEGER,
            machine_id TEXT,
            activity_type TEXT,
            part_id INTEGER,
            queue TEXT
        )
        """)
        realdb.commit()

    # inserting data
    with sqlite3.connect(database_path) as realdb: 
                realdb.executemany(f"""
                INSERT INTO {table_name} (timestamp, machine_id, activity_type, part_id, queue)
                VALUES (?, ?, ?, ?, ?)""", new_data)
                realdb.commit()

### Input data

In [6]:
# input data
file_path = r'log_files/5s_Model_Tracelog.txt'    # also has the name of file in it.
database_path = r'databases/real_5s_closed_distribution_db.db'            # # also has the name of database in it.

table_name = 'tracelog'
dbwrite_real_arenatracelog(table_name,file_path,database_path)

### Additional functions

In [187]:
# delete table
with sqlite3.connect(database_path) as realdb:
    realdb.execute(f"DROP TABLE IF EXISTS {table_name}")
    realdb.commit()

In [7]:
# list of tables
with sqlite3.connect(database_path) as realdb:
    table_names = realdb.execute("SELECT name from sqlite_master where type='table'").fetchall()

print(table_names)

[('tracelog',)]


In [9]:
# fetching a specific column
with sqlite3.connect(database_path) as DB:
    activity=DB.execute(f"SELECT {'activity_type'} FROM {table_name}").fetchall()

print(len(activity))

309


## Arena KPILog Database Interface

### Main fnuction to read and insert data

In [201]:
def dbwrite_real_arenakpilog(table_name,file_path,database_path):
    # reading the text file as a space separated csv file into pandas dataframe.
    data = pd.read_csv(file_path, delimiter='\s+', header=None, index_col=False)

    # data preparation 
    # indexing of dataframe starts at 0.
    part_id=data[0].values.astype(int).tolist()
    cycle_time=data[1].values.astype(int).tolist()

    # zip all the list created above into a single list
    new_data=list(zip(part_id, cycle_time))

    # create table. Also creates database if it doesnt exist.
    with sqlite3.connect(database_path) as realdb:
        realdb.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            kpi_id INTEGER PRIMARY KEY,
            part_id INTEGER,
            cycle_time INTEGER
        )
        """)
        realdb.commit()

    # inserting data
    with sqlite3.connect(database_path) as realdb: 
                realdb.executemany(f"""
                INSERT INTO {table_name} (part_id, cycle_time)
                VALUES (?, ?)""", new_data)
                realdb.commit()

### Input data

In [204]:
# input data
file_path = r'C:\Users\HP\Desktop\5s_Model_KPILog.txt'    # also has the name of file in it.
database_path = r'C:\Users\HP\Desktop\realdb.db'            # # also has the name of database in it.
table_name = 'kpilog'
dbwrite_real_arenakpilog(table_name,file_path,database_path)

### Additional function

In [203]:
# delete table
with sqlite3.connect(database_path) as realdb:
    realdb.execute(f"DROP TABLE IF EXISTS {table_name}")
    realdb.commit()

In [205]:
# list of tables
with sqlite3.connect(database_path) as realdb:
    table_names = realdb.execute("SELECT name from sqlite_master where type='table'").fetchall()

print(table_names)

[('tracelog',), ('kpilog',)]


In [209]:
# fetching a specific column
with sqlite3.connect(database_path) as DB:
    cycle_time=DB.execute(f"SELECT {'cycle_time'} FROM {table_name}").fetchall()